In [1]:
import tensorflow as tf
import plotly.express as px
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications.efficientnet import preprocess_input

In [72]:
directory = 'images/final/'
width = 128
height = 128
channels = 4
image_size = (width, height)
input_shape = (width, height, channels)
batch_size = 32

In [3]:
def plot_images(images, labels):
    fig = px.imshow(images, width=width, height=height)
    fig.update_layout(
        title=f"Label: {labels}",
        width=300,
        height=300,
    )
    fig.show()

In [4]:
def plot_n_images(ds, n):
    for image, label in ds.take(1):
        for i in range(n):
            print(image[i][0][0])
            print(image[i].shape)
            plot_images(image[i], class_names[int(tf.argmax(tf.reshape(label[i], [-1, 1]), axis=0))])

In [73]:
train_ds, val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    batch_size=batch_size,
    validation_split=0.2,
    subset="both",
    labels="inferred",
    label_mode="categorical",
    image_size=image_size,
    crop_to_aspect_ratio=True,
    interpolation="bilinear",
    color_mode="rgba",
    shuffle=True,
    seed=905,
)

class_names = train_ds.class_names

Found 13894 files belonging to 905 classes.
Using 11116 files for training.
Using 2778 files for validation.


In [74]:
plot_n_images(train_ds, 1)

tf.Tensor([0. 0. 0. 0.], shape=(4,), dtype=float32)
(128, 128, 4)


In [75]:
flip_and_rotate = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1, fill_mode="constant"),
    # tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
])

def preprocess_image(image):
    # print(image.shape)
    alpha_channel = image[:, :, :, 3]
    rgb_channels = image[:, :, :, :3]
    alpha_bool = alpha_channel > 0
    alpha_bool = tf.expand_dims(alpha_bool, axis=-1)
    # print(alpha_bool.shape)
    # print(rgb_channels.shape)
    rgb_channels = tf.where(alpha_bool, rgb_channels, 0)
    
    # Normalize the pixel values to be between 0 and 1
    image = preprocess_input(rgb_channels)
    
    return image

def prepare(ds, shuffle=False, augment=False):
    ds = ds.map(lambda x, y: (preprocess_image(x), y), num_parallel_calls=tf.data.AUTOTUNE)

    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(lambda x, y: (flip_and_rotate(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
    
    # Use buffered prefecting on all datasets
    if shuffle:
        ds = ds.shuffle(1000)

    # ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)

    return ds

In [76]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)

channels = 3
input_shape = (width, height, channels)

In [77]:
plot_n_images(train_ds, 5)

In [12]:
# Define the EfficientNetB4 model
base_model = EfficientNetB4(include_top=False, input_shape=(width, height, 3), pooling='avg', weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Define the model inputs
inputs = tf.keras.Input(shape=(width, height, 4), batch_size=batch_size)

# Preprocess the inputs
if channels == 4:
    # Convert the image to RGB and remove the alpha channel
    x = tf.cond(tf.equal(tf.shape(inputs)[-1], 4),
                lambda: inputs[..., :3],
                lambda: inputs)

    # Convert the image to float32
    x = tf.cast(x, tf.float32)

    # Replace black pixels with white pixels
    mask = tf.reduce_all(tf.equal(x, [0, 0, 0]), axis=-1)
    mask = tf.expand_dims(mask, axis=-1)
    x = tf.where(mask, tf.ones_like(x), x)
else:
    x = inputs

x = base_model(x, training=False)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)

# Define the model
model = tf.keras.Model(inputs, x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['top_k_categorical_accuracy'])

TypeError: Exception encountered when calling layer "tf.cond_1" (type TFOpLambda).

To be compatible with tf.function, Python functions must return zero or more Tensors or ExtensionTypes or None values; in compilation of <function <lambda> at 0x00000271F34A7C70>, found return value of type KerasTensor, which is not a Tensor or ExtensionType.

Call arguments received by layer "tf.cond_1" (type TFOpLambda):
  • pred=tf.Tensor(shape=(), dtype=bool)
  • true_fn=<function <lambda> at 0x00000271F34A7C70>
  • false_fn=<function <lambda> at 0x00000271F34A7B50>
  • name=None

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(32, 128, 128, 3)]       0         
                                                                 
 efficientnetb4 (Functional)  (None, 1792)             17673823  
                                                                 
 dense (Dense)               (32, 512)                 918016    
                                                                 
 dropout (Dropout)           (32, 512)                 0         
                                                                 
 dense_1 (Dense)             (32, 256)                 131328    
                                                                 
 dropout_1 (Dropout)         (32, 256)                 0         
                                                                 
 dense_2 (Dense)             (32, 905)                 232585

In [10]:
model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\valen